In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

In [19]:
# from nba_api.stats.endpoints import leaguegamefinder

conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

seasons = []
for i in list(range(1983,2025)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]

## Takes only 1:20 to pull
gamesl = []
for i in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=i).get_data_frames()[0]
    gamesl.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(gamesl,axis=0)
gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])

## Takes only 1:07
playersl = []
for i in seasons:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    playersl.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(playersl,axis=0)

##### THIS PROCESSES AND RECREATES THE MASTER TABLES 
### DROP TABLES
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS mastergames;")
cursor.execute("DROP TABLE IF EXISTS masterplayers;")

conn.commit()

gamehistory = allintwo.clean_column_names(gamehistory)
allplayers = allintwo.clean_column_names(allplayers)

allintwo.create_table(conn,'mastergames',gamehistory)
allintwo.create_table(conn,'masterplayers',allplayers)

allintwo.insert_dataframe_to_rds(conn,
    gamehistory,
    table_name='mastergames'
)

allintwo.insert_dataframe_to_rds(conn,
    allplayers,
    table_name='masterplayers'
)


Connected to RDS PostgreSQL database


Master table mastergames created successfully.
Master table masterplayers created successfully.
Data inserted successfully into mastergames table.
Data inserted successfully into masterplayers table.


In [ ]:
# Call the function, passing the existing connection
df = allintwo.fetch_table_to_dataframe(conn, 'mastergames')


Data fetched successfully from mastergames table.


,seasonid,teamid,teamabbreviation,teamname,gameid,gamedate,matchup,wl,min,pts,...,ftpct,oreb,dreb,reb,ast,stl,blk,tov,pf,plusminus
0,12024,1610612763,MEM,Memphis Grizzlies,0012400071,2024-10-18,MEM vs. MIA,L,239,109,...,0.870,12.0,34.0,46,28,10.0,5,23,24,-5.0
1,12024,1610612752,NYK,New York Knicks,0012400068,2024-10-18,NYK @ WAS,L,239,117,...,0.816,8.0,30.0,38,22,7.0,6,15,17,-1.0
2,12024,1610612761,TOR,Toronto Raptors,0012400069,2024-10-18,TOR @ BKN,W,240,116,...,0.680,7.0,38.0,45,34,9.0,7,18,26,4.0
3,12024,1610612762,UTA,Utah Jazz,0012400073,2024-10-18,UTA @ POR,L,239,86,...,0.700,12.0,32.0,44,16,5.0,10,22,18,-38.0
4,12024,1610612757,POR,Portland Trail Blazers,0012400073,2024-10-18,POR vs. UTA,W,241,124,...,0.900,20.0,36.0,56,34,16.0,10,13,21,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123097,21983,1610612762,UTH,Utah Jazz,0028300009,1983-10-28,UTH @ DEN,L,240,125,...,0.781,10.0,27.0,37,32,6.0,8,7,30,NaN
123098,21983,1610612744,GOS,Golden State Warriors,0028300004,1983-10-28,GOS @ SEA,W,240,110,...,0.703,21.0,26.0,47,26,11.0,2,19,34,NaN
123099,21983,1610612746,SDC,San Diego Clippers,0028300007,1983-10-28,SDC @ POR,L,240,93,...,0.750,7.0,24.0,31,19,5.0,5,18,31,NaN
123100,21983,1610612737,ATL,Atlanta Hawks,0028300005,1983-10-28,ATL @ NJN,L,240,108,...,0.684,15.0,18.0,33,14,10.0,5,17,30,NaN
